Before running, install required packages:

In [1]:
! pip install numpy sklearn torchvision tensorboardX


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 125.4/125.4 kB 822.2 kB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2343 sha256=e2c778d99553f409cc04023270cbecc3c50355fb065d131f92b1021d3076aa78
  Stored in directory: c:\users\daani\appdata\local\pip\cache\wheels\03\8b\6f\9f13c705de81a6b351b718b3cf917e41ad7c0933c8630d4dd4
Successfully built sklearn



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


---

In [9]:
import numpy as np
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from torchvision import datasets, transforms
import urllib
import zipfile
from tensorboardX import SummaryWriter
from datetime import datetime
from PIL import ImageFile

In [10]:
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [11]:
# COMMENT THIS OUT IF YOU USE YOUR OWN DATA.
# Download example data into ./data/image-data (4 image files, 2 for "dog", 2 for "cat").
# url = "https://github.com/jrieke/traingenerator/raw/main/data/fake-image-data.zip"
# zip_path, _ = urllib.request.urlretrieve(url)
# with zipfile.ZipFile(zip_path, "r") as f:
#     f.extractall("data")

# Setup

In [12]:
# INSERT YOUR DATA HERE
# Expected format: One folder per class, e.g.
# train
# --- dogs
# |   +-- lassie.jpg
# |   +-- komissar-rex.png
# --- cats
# |   +-- garfield.png
# |   +-- smelly-cat.png
#
# Example: https://github.com/jrieke/traingenerator/tree/main/data/image-data
train_data = "train/"  # required
val_data = "train/"    # optional
test_data = "test/"                # optional

In [13]:
# Set up logging.
experiment_id = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
writer = SummaryWriter(logdir=f"logs/{experiment_id}")

# Preprocessing

In [14]:
# Set up scaler.
scaler = sklearn.preprocessing.StandardScaler()

In [15]:
def preprocess(data, name):
    if data is None:  # val/test can be empty
        return None
    # Read image files to pytorch dataset (only temporary).
    transform = transforms.Compose([
        transforms.Resize(28), 
        transforms.CenterCrop(28), 
        transforms.ToTensor()
    ])
    data = datasets.ImageFolder(data, transform=transform)

    # Convert to numpy arrays.
    images_shape = (len(data), *data[0][0].shape)
    images = np.zeros(images_shape)
    labels = np.zeros(len(data))
    for i, (image, label) in enumerate(data):
        images[i] = image
        labels[i] = label
    
    # Flatten.
    images = images.reshape(len(images), -1)

    # Scale to mean 0 and std 1.
    if name == "train":
        scaler.fit(images)
    images = scaler.transform(images)

    # Shuffle train set.
    if name == "train":
        images, labels = sklearn.utils.shuffle(images, labels)

    return [images, labels]

In [16]:
processed_train_data = preprocess(train_data, "train")
processed_val_data = preprocess(val_data, "val")
processed_test_data = preprocess(test_data, "test")

# Model

In [17]:
model = KNeighborsClassifier()

# Training

In [18]:
def evaluate(data, name):
    if data is None:  # val/test can be empty
        return

    images, labels = data
    acc = model.score(images, labels)
    print(f"{name + ':':6} accuracy: {acc}")
    writer.add_scalar(f"{name}_accuracy", acc)

In [19]:
# Train on train_data.
model.fit(*processed_train_data)

KNeighborsClassifier()

In [20]:
# Evaluate on all datasets.
evaluate(processed_train_data, "train")
evaluate(processed_val_data, "val")
evaluate(processed_test_data, "test")

train: accuracy: 0.76
val:   accuracy: 0.76
test:  accuracy: 0.6595744680851063


In [21]:
import pickle
filename = 'KNN.sav'
pickle.dump(model, open(filename, 'wb'))